#### Index Creation Cost Model -  Online Ridge Regression

In [1]:
import logging
import datetime
import os
import subprocess
import uuid

import pyodbc
import sys
import random
import pandas as pd
import time
import os
from tqdm import tqdm
import logging
import re
import json
import xml.etree.ElementTree as ET
import itertools
import math
from collections import defaultdict
from tqdm import tqdm
import pickle
import numpy as np

%load_ext autoreload
%autoreload 2

import IPython
notebook_path = IPython.get_ipython().starting_dir
target_subdirectory_path = os.path.abspath(os.path.join(os.path.dirname(notebook_path), 'database'))
sys.path.append(target_subdirectory_path)
from utils import *

#### Generate all possible indexes for each table.

In [2]:
"""
    Function for generating all possible configurations (i.e. subsets of indixes)
"""
def generate_all_configurations(connection, MAX_COLS=2, with_includes=False, verbose=False):
    # first, generate all possible indices
    tables = get_all_tables(connection)
    all_indices = {} 
    # tqdm bar around table loop
    for table_name, table in tqdm(tables.items(), desc="Generating all indices for table:"):
        columns = table.get_columns()
        if verbose:
            print(f"Table --> {table_name} with columns --> {columns}")
        # get all possible permutations of columns, up to MAX_KEY_COLS columns
        for num_columns in range(1, MAX_COLS+1):
            col_permutations = list(itertools.permutations(columns, num_columns))
            # also generate permutations of columns with include columns
            for cp in col_permutations:
                if with_includes:
                    # get columns not in cp
                    include_columns = list(set(columns) - set(cp))
                    # get all comnbination of include columns on remaining columns
                    include_col_combinations = list(itertools.combinations(include_columns, MAX_COLS-num_columns))
                    for icp in include_col_combinations:
                        index_id = get_index_id(cp, table_name, include_columns)
                        if index_id not in all_indices:
                            index_size = get_estimated_index_size(connection, table_name, list(cp) + include_columns)
                            # create index object
                            all_indices[index_id] = Index(table_name, index_id, cp, index_size, tuple(icp))
                else:
                    index_id = get_index_id(cp, table_name)
                    if index_id not in all_indices:
                        index_size = get_estimated_index_size(connection, table_name, list(cp))
                        # create index object
                        all_indices[index_id] = Index(table_name, index_id, cp, index_size)

    print(f"Total number of indices generated: {len(all_indices)}, total estimated size: {sum([i.size for i in all_indices.values()]):.2f} Mb")
   
    return all_indices, tables


In [3]:
connection = start_connection()

all_indices, tables = generate_all_configurations(connection, MAX_COLS=3, with_includes=False)


# get all columns
all_columns, num_columns = get_all_columns(connection)

columns_to_idx = {}
i = 0
for table_name, columns in all_columns.items():
    for column in columns:
        columns_to_idx[column] = i
        i += 1

idx_to_columns = {v: k for k, v in columns_to_idx.items()}  

close_connection(connection)

Generating all indices for table:: 100%|██████████| 8/8 [00:00<00:00, 484.46it/s]

Total number of indices generated: 5585, total estimated size: 824332.61 Mb


#### Create feature vector for each index:

Feature vector consists of two parts: (1) Encoding of index and include columns, (2) Table stats

In [4]:
def get_table_stats(connection, table_name):
    cursor = connection.cursor()
    cursor.execute(f"""
                        SELECT SUM(row_count) as total_rows, SUM(used_page_count) * 8 / 1024.0 as size_mb
                        FROM sys.dm_db_partition_stats
                        WHERE object_id = OBJECT_ID('{table_name}')
                    """)
    row_count, table_size_mb = cursor.fetchone()
    table_size_mb = float(table_size_mb) if table_size_mb is not None else 0.0

    cursor.execute(f"""
            SELECT AVG(avg_fragmentation_in_percent)
            FROM sys.dm_db_index_physical_stats(DB_ID(), OBJECT_ID('{table_name}'), NULL, NULL, 'LIMITED')
        """)
    avg_fragmentation = cursor.fetchone()[0]
    avg_fragmentation = float(avg_fragmentation) if avg_fragmentation is not None else 0.0

    #cursor.execute("SELECT cpu_count FROM sys.dm_os_sys_info")
    #cpu_count = cursor.fetchone()[0]

    num_columns = len(all_columns[table_name])
    return (num_columns, row_count, table_size_mb, avg_fragmentation)
    

def column_feature_encoding(index, c=10):
    index_columns_encoding = np.zeros(len(columns_to_idx), dtype=float)
    # encoding for index columns
    for j, column_name in enumerate(index.index_columns):
        column_position_in_index = j
        index_columns_encoding[columns_to_idx[column_name]] = 1/(c**column_position_in_index)
    
    if len(index.include_columns) > 0:
        # encoding for include columns
        include_columns_encoding = np.zeros(len(columns_to_idx), dtype=float)
        for j, column_name in enumerate(index.include_columns):
            include_columns_encoding[columns_to_idx[column_name]] = 1

        # concatenate the two context vectors
        columns_encoding = np.hstack((index_columns_encoding, include_columns_encoding))
        return columns_encoding
    else:

        return index_columns_encoding 
    

def create_feature_vectors(all_indices, connection):

    print(f"Obtain table stats...")    
    table_stats = {}
    for index in all_indices.values():
        table_name = index.table_name
        if table_name not in table_stats:
            table_stats[table_name] = get_table_stats(connection, table_name)
    
    
    print(f"Creating feature vectors...")
    feature_vectors = {}
    for index in all_indices.values():
        columns_encoding = column_feature_encoding(index)
        index_size_mb = index.size
        num_columns, row_count, table_size_mb, avg_fragmentation = table_stats[index.table_name]
        table_features = np.array([num_columns, row_count, table_size_mb/index_size_mb, avg_fragmentation])
        feature_vectors[index.index_id] = np.concatenate((table_features, columns_encoding))
        
    return feature_vectors  



In [5]:
connection = start_connection()

feature_vectors = create_feature_vectors(all_indices, connection)

close_connection(connection)

Obtain table stats...
Creating feature vectors...


In [6]:
# group the indexes by table
indexes_by_table = defaultdict(list)
for index in all_indices.values():
    indexes_by_table[index.table_name].append(index)


# create train test splits for the indexes, from each table pick 10 indexes for training
train_indexes = []
test_indexes = []
split_idx = 20
for table_name, indexes in indexes_by_table.items():
    random.shuffle(indexes)
    train_indexes.extend(indexes[:split_idx])
    test_indexes.extend(indexes[split_idx:])

random.shuffle(train_indexes) 
random.shuffle(test_indexes)

print(f"Number of training indexes: {len(train_indexes)}, number of test indexes: {len(test_indexes)}")

Number of training indexes: 155, number of test indexes: 5430


In [7]:
# measure actual creation times of training indexes
connection = start_connection()

index_creation_cost = {}
for index in tqdm(train_indexes, desc="Measuring index creation cost:"):
    index_creation_cost[index.index_id] = create_nonclustered_index_object(connection, index)
    drop_noncluster_index_object(connection, index)

close_connection(connection)

Measuring index creation cost::   0%|          | 0/155 [00:00<?, ?it/s]

Measuring index creation cost:: 100%|██████████| 155/155 [09:56<00:00,  3.85s/it]


#### Online Ridge Regression Model

In [8]:
#from sklearn.preprocessing import StandardScaler

class Model:
    def __init__(self, feature_vectors, all_indices, lambda_reg=0.1, epsilon=1e-8):
        self.feature_vectors = feature_vectors
        self.all_indices = all_indices
        self.feature_dims = feature_vectors[list(feature_vectors.keys())[0]].shape[0]
        self.V = lambda_reg * np.eye(self.feature_dims)
        self.b = np.zeros(self.feature_dims)
        self.theta = np.zeros(self.feature_dims)
        self.lambda_reg = lambda_reg
        self.epsilon = epsilon
        self.loss_history = []
        #self.scaler = StandardScaler()
        #self.normalize_features()

    #def normalize_features(self):
    #    all_features = np.array(list(self.feature_vectors.values()))
    #    self.scaler.fit(all_features)
    #    for key in self.feature_vectors:
    #        self.feature_vectors[key] = self.scaler.transform([self.feature_vectors[key]])[0]

    def update(self, index, cost, verbose=False):
        x = self.feature_vectors[index.index_id]    
        y = cost
        self.V += np.outer(x, x)
        self.b += y * x
        # add small epsilon to diagonal of V for conditioning
        self.theta = np.linalg.solve(self.V + self.epsilon*np.eye(self.feature_dims), self.b)
        loss, y_pred = self.compute_loss(x, y)
        if verbose:
            print(f"Update for index: {index.index_id}, actual cost: {cost}, predicted cost: {y_pred:.3f}, loss incurred: {loss}")

    def predict(self, x):
        return np.dot(self.theta, x)

    def compute_loss(self, x, y):
        y_pred = self.predict(x)
        mse = (y - y_pred)**2
        reg = self.lambda_reg * np.dot(self.theta, self.theta)
        loss = mse + reg
        self.loss_history.append(loss)
        return loss, y_pred    
        

#### Train the model

In [9]:
model = Model(feature_vectors, all_indices, lambda_reg=0.01)

In [10]:
for index in train_indexes:
    cost = index_creation_cost[index.index_id]
    model.update(index, cost, verbose=True)  

Update for index: IX_nation_n_nationkey_n_name_n_comment, actual cost: 0.0, predicted cost: 0.000, loss incurred: 0.0
Update for index: IX_supplier_s_nationkey_s_address_s_suppkey, actual cost: 0.021, predicted cost: 0.021, loss incurred: 6.03864462275018e-13
Update for index: IX_supplier_s_suppkey_s_name, actual cost: 0.009, predicted cost: 0.009, loss incurred: 6.33396695730856e-07
Update for index: IX_partsupp_ps_partkey_ps_suppkey_ps_availqty, actual cost: 0.808, predicted cost: 0.808, loss incurred: 6.379744199034224e-07
Update for index: IX_orders_o_custkey_o_totalprice_o_comment, actual cost: 3.6, predicted cost: 3.599, loss incurred: 0.0018897478000131699
Update for index: IX_lineitem_l_shipdate_l_suppkey, actual cost: 10.509, predicted cost: 10.511, loss incurred: 0.023134156961332836
Update for index: IX_part_p_size, actual cost: 0.355, predicted cost: 0.353, loss incurred: 0.025243663567726487
Update for index: IX_part_p_comment_p_mfgr_p_partkey, actual cost: 0.433, predicte

#### Make predictions for some test indexes.

In [11]:
# make predictions for some test indexes and compare with actual costs
connection = start_connection()

remove_all_nonclustered_indexes(connection)

test_indexes_sample = random.sample(test_indexes, 20)
test_index_creation_cost = {}
for index in tqdm(test_indexes_sample, desc="Measuring index creation cost:"):
    test_index_creation_cost[index.index_id] = create_nonclustered_index_object(connection, index)
    drop_noncluster_index_object(connection, index)

close_connection(connection)

All 6 nonclustered indexes removed.


Measuring index creation cost:: 100%|██████████| 20/20 [05:11<00:00, 15.60s/it]


In [12]:
test_losses = []
for index in test_indexes_sample:
    x = model.feature_vectors[index.index_id]
    y = test_index_creation_cost[index.index_id]
    loss, y_pred = model.compute_loss(x, y)
    test_losses.append(loss)
    print(f"Index: {index.index_id}, actual cost: {y:.3f}, predicted cost: {y_pred:.3f}, loss: {loss:.3f}")


Index: IX_lineitem_l_receiptdate_l_suppkey_l_shipinstruct, actual cost: 14.719, predicted cost: 23.710, loss: 93.627
Index: IX_lineitem_l_tax_l_shipmode, actual cost: 15.668, predicted cost: 14.491, loss: 14.175
Index: IX_lineitem_l_shipinstruct_l_tax_l_shipdate, actual cost: 27.873, predicted cost: 33.152, loss: 40.656
Index: IX_lineitem_l_shipdate_l_extendedprice_l_partkey, actual cost: 13.617, predicted cost: 9.366, loss: 30.857
Index: IX_lineitem_l_suppkey_l_tax_l_extendedprice, actual cost: 18.042, predicted cost: 41.453, loss: 560.858
Index: IX_lineitem_l_quantity_l_comment_l_linestatus, actual cost: 23.898, predicted cost: 20.278, loss: 25.892
Index: IX_lineitem_l_discount_l_quantity_l_shipmode, actual cost: 29.273, predicted cost: 18.406, loss: 130.881
Index: IX_part_p_name_p_brand_p_mfgr, actual cost: 0.732, predicted cost: 0.619, loss: 12.803
Index: IX_lineitem_l_partkey_l_discount_l_quantity, actual cost: 16.481, predicted cost: 13.950, loss: 19.194
Index: IX_lineitem_l_ship